In [2]:
import pandas as pd
import numpy as np
import tabula
import pymongo
import json
import os

from dotenv import load_dotenv
load_dotenv()

dbuser = "backend"
dbpass = os.environ["backend"]
dbconn = os.environ["connpython"]
origin = os.environ["origin"]

tabs = tabula.read_pdf(origin, pages="7", lattice=True, multiple_tables=True, java_options="-Dfile.encoding=UTF8",)
print(f"Found {len(tabs)} tables")

#---

df = tabs[1]
cid = list(df.columns.values)[0]
cname = list(df.columns.values)[1]

df.columns.values[0] = "Codigo"
df.columns.values[1] = "Atividade"
df.columns.values[2] = "Valor"
df.columns.values[3] = "Porc"
df["Classe"] = cname
df["Subclasse"] = ""

df['Codigo'] = df['Codigo'].fillna("")
df['Valor'] = df['Valor'].fillna("")
df['Porc'] = df['Porc'].fillna("")

doc = df.to_json(orient = "records")
doc = json.loads(doc)

conn = dbconn.replace("<username>", dbuser).replace("<password>", dbpass)
rs = pymongo.MongoClient(conn)

db = rs.legalfees
col = db.fees
col.insert_many(doc)

display(df)

Found 3 tables


,Codigo,Atividade,Valor,Porc,Classe,Subclasse
0,5.1,Pedido de falência e acompanhamento até a decr...,"R$ 4.335,88",20%,"ATIVIDADES EM MATÉRIA DE INSOLVÊNCIA, FALÊNCIA...",
1,5.2,"Ação de restituição e ação reivindicatória, at...","R$ 4.335,88",20%,"ATIVIDADES EM MATÉRIA DE INSOLVÊNCIA, FALÊNCIA...",
2,5.3,Pedido de recuperação de empresa,"R$ 7.687,75",2% a 10%,"ATIVIDADES EM MATÉRIA DE INSOLVÊNCIA, FALÊNCIA...",
3,5.4,Pedido de declaração de insolvência,"R$ 3.613,24",20%,"ATIVIDADES EM MATÉRIA DE INSOLVÊNCIA, FALÊNCIA...",
4,5.5,Habilitação tempestiva ou retardatária e diver...,"R$ 3.613,24",20%,"ATIVIDADES EM MATÉRIA DE INSOLVÊNCIA, FALÊNCIA...",
5,5.6,Representação do falido (sobre o montante do p...,"R$ 7.226,48",20%,"ATIVIDADES EM MATÉRIA DE INSOLVÊNCIA, FALÊNCIA...",
6,5.7,Representação do devedor insolvente (sobre o m...,"R$ 7.226,48",20%,"ATIVIDADES EM MATÉRIA DE INSOLVÊNCIA, FALÊNCIA...",
7,5.8,Representação do administrador judicial na fal...,"R$ 8.671,79",10%,"ATIVIDADES EM MATÉRIA DE INSOLVÊNCIA, FALÊNCIA...",
